# Setup

In [1]:
# Weights and Biases
!pip install -q wandb
# Tensorflow
!pip install -q tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 552.3 kB/s eta 0:00:00


In [2]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tensorflow import square, reduce_mean
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.math import multiply
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError
from math import log

In [3]:
# If running in colab, insert your wandb key here

#import config
#Erlend
#wandb.login(key=config.erlend_key)
# Hjalmar
wandb.login(key="b47bcf387a0571c5520c58a13be35cda8ada0a99")


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Load, split and normalize data

### Load data

In [7]:
google_colab = True

if google_colab:
    import tensorflow as tf
    # Pring info
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)
    
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

    # Code to read csv file into Colaboratory:
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    id = "16SwdE7VcT6UCzVNQmtOM3914nwWJMOmN"
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('2020_2022_moneyness_filtere.csv')  
    df_read = pd.read_csv('2020_2022_moneyness_filtere.csv')
else:
    file = "../data/processed_data/2020_2022_moneyness_filtere.csv"
    df_read = pd.read_csv(file)

display(df_read)

Fri Mar 24 19:55:43 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

,Unnamed: 0.1,Unnamed: 0,Quote_date,Price,Underlying_last,Strike,TTM,R,Moneyness
0,6701729,6701729,2020-01-02,1755.645,3258.14,1500.0,1,1.53,2.172093
1,6701730,6701730,2020-01-02,1655.955,3258.14,1600.0,1,1.53,2.036338
2,6701731,6701731,2020-01-02,1556.195,3258.14,1700.0,1,1.53,1.916553
3,6701732,6701732,2020-01-02,1456.210,3258.14,1800.0,1,1.53,1.810078
4,6701733,6701733,2020-01-02,1406.200,3258.14,1850.0,1,1.53,1.761157
...,...,...,...,...,...,...,...,...,...
5185540,12459146,12459146,2022-12-30,415.150,3839.81,4500.0,1085,4.22,0.853291
5185541,12459147,12459147,2022-12-30,375.050,3839.81,4600.0,1085,4.22,0.834741
5185542,12459148,12459148,2022-12-30,337.350,3839.81,4700.0,1085,4.22,0.816981
5185543,12459149,12459149,2022-12-30,302.650,3839.81,4800.0,1085,4.22,0.799960


In [8]:
df = df_read
del df_read

# Group the data by Quote Date and calculate the mean for Underlying Price
df_agg = df.groupby('Quote_date').mean().reset_index()

# Values to returns
df_agg["Underlying_return"] = df_agg["Underlying_last"].pct_change()

lags = 10

# Add the Underlying Price Lag column
for i in range(1, lags + 1):
    df_agg['Underlying_' + str(i)] = df_agg['Underlying_return'].shift(i)

df = pd.merge(df, df_agg[['Quote_date', "Underlying_return"] + ['Underlying_' + str(i) for i in range(1, lags + 1)]], on='Quote_date', how='left')

# Filter df between 2021-01-01 and 2022-12-31
df = df[(df["Quote_date"] >= "2021-01-01") & (df["Quote_date"] <= "2022-12-31")]

### Format input data

In [9]:
# Format settings
max_timesteps = lags
moneyness = False
bs_vars = ['Moneyness', 'TTM', 'R'] if moneyness else ['Underlying_last', 'Strike', 'TTM', 'R']
underlying_lags = ['Underlying_last'] + [f'Underlying_{i}' for i in range (1, max_timesteps)]

def create_rw_dataset(window_number = 0, df = df):
    '''Creates dataset for a single rolling window period offsett by the window number'''

    # Create train, validation and test set split points
    train_start = datetime(2021,1,1) + relativedelta(months=window_number)
    val_start = train_start + relativedelta(months=11)
    test_start = val_start + relativedelta(months=1)
    test_end = test_start + relativedelta(months=1)
    train_start = str(train_start.date())
    val_start = str(val_start.date())
    test_start = str(test_start.date())
    test_end = str(test_end.date())

    # Split train and validation data
    df_train = df[(df['Quote_date'] >= train_start) & (df['Quote_date'] < val_start)]
    df_val = df[(df['Quote_date'] >= val_start) & (df['Quote_date'] < test_start)]
    df_test = df[(df['Quote_date'] >= test_start) & (df['Quote_date'] < test_end)]

    del df
    # Extract target values
    train_y = (df_train['Price'] / df_train['Strike']).to_numpy() if moneyness else df_train['Price'].to_numpy()
    val_y = (df_val['Price'] / df_val['Strike']).to_numpy() if moneyness else df_val['Price'].to_numpy()
    test_y = (df_test['Price'] / df_test['Strike']).to_numpy() if moneyness else df_test['Price'].to_numpy()

    # If usining moneyness, extract strike
    if moneyness:
        train_strike = df_train['Strike'].to_numpy()
        val_strike = df_val['Strike'].to_numpy()
        test_strike = df_test['Strike'].to_numpy()

    # Convert dataframes to numpy arrays
    train_x = [df_train[underlying_lags].to_numpy(), df_train[bs_vars].to_numpy()]
    val_x = [df_val[underlying_lags].to_numpy(), df_val[bs_vars].to_numpy()]
    test_x = [df_test[underlying_lags].to_numpy(), df_test[bs_vars].to_numpy()]

    del df_train
    del df_val

    # Scale features based on training set
    underlying_scaler = MinMaxScaler()
    train_x[0] = underlying_scaler.fit_transform(train_x[0])
    val_x[0] = underlying_scaler.transform(val_x[0])
    test_x[0] = underlying_scaler.transform(test_x[0])

    bs_scaler = MinMaxScaler()
    train_x[1] = bs_scaler.fit_transform(train_x[1])
    val_x[1] = bs_scaler.transform(val_x[1])
    test_x[1] = bs_scaler.transform(test_x[1])


    # Shuffle training set
    np.random.seed(0)
    shuffle = np.random.permutation(len(train_x[0]))
    train_x = [train_x[0][shuffle], train_x[1][shuffle]]
    train_y = train_y[shuffle]
    if moneyness:
        train_strike = train_strike[shuffle]

    # Reshape data to fit LSTM
    train_x = [train_x[0].reshape(len(train_x[0]), max_timesteps, 1), train_x[1]]
    val_x = [val_x[0].reshape(len(val_x[0]), max_timesteps, 1), val_x[1]]
    test_x = [test_x[0].reshape(len(test_x[0]), max_timesteps, 1), test_x[1]]

    print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
    print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')
    print(f'Test shape: {test_x[0].shape}, {test_x[1].shape}')

    if moneyness:
        return train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike,
    return train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test

# Create the dataset for the first rolling window period
if moneyness:
    train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike = create_rw_dataset()
else:
    train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset()

Train shape: (1655728, 10, 1), (1655728, 4)
Val shape: (184525, 10, 1), (184525, 4)
Test shape: (165072, 10, 1), (165072, 4)


# Model construction

In [10]:
def create_model(config):
    '''Builds an LSTM-MLP model of minimum 2 layers sequentially from a given config dictionary'''

    # Input layers
    underlying_history = Input((config.LSTM_timesteps,1))
    bs_vars = Input((config.Num_features,))

    # LSTM layers
    model = Sequential()

    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        input_shape = (config.LSTM_timesteps, 1),
        return_sequences = True
    ))

    for _ in range(config.LSTM_layers - 2):
        model.add(LSTM(
            units = config.LSTM_units,
            activation = tanh,
            return_sequences = True
        ))
    
    model.add(LSTM(
        units = config.Interface_units,
        activation = tanh,
        return_sequences = False
    ))

    # MLP layers
    layers = Concatenate()([model(underlying_history), model(underlying_history), model(underlying_history), model(underlying_history), model(underlying_history), bs_vars])
    
    for _ in range(config.MLP_layers - 1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU()(layers)

    output = Dense(1, activation='relu')(layers)

    # Exponential decaying learning rate
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x[0])/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    # Compile model
    model = Model(inputs=[underlying_history, bs_vars], outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    model.summary()
    return model

# Hyperparameter search setup

In [ ]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'bayes',
    'name': 'LSTM-MLP v5.0',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'LSTM_units': {
            'values': [4, 8, 16, 32]},
        'Interface_units': {
            'values': [4, 8, 16, 32]},
        'MLP_units': {
            'values': [50, 200, 600]},
        'LSTM_timesteps': {
            'values': [10, 20, 40]},
        'LSTM_layers': {
            'distribution': 'int_uniform',
            'max': 8, 'min': 2},
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 8, 'min': 2},
        'Bn_momentum': {
            'values': [0.1, 0.4, 0.7, 0.99]},
        'Lr': {
            'distribution': 'log_uniform',
            'max': log(0.1), 'min': log(0.0001)},
        'Lr_decay': {
            'distribution': 'log_uniform',
            'max': log(1), 'min': log(0.8)},        
        'Minibatch_size': {
            'value': 4096},
        'Min_delta': {
            'value': 0.01 if moneyness else 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value': 3 if moneyness else 4},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing') 
#sweep_id = '98bxt6oq'

wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.
wandb: WARNING   Violation 2. Lr_decay uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: r6jvzqce
Sweep URL: https://wandb.ai/avogadro/Deep%20learning%20for%20option%20pricing%20-%20test%20area/sweeps/r6jvzqce


# Run hyperparameter search

In [11]:
#WIP
class MSE_LossCallback(Callback):
    def __init__(self, train_x, train_y, train_strike, val_x, val_y, val_strike):
        self.train_x = train_x
        self.train_y = train_y
        self.train_strike = train_strike
        self.val_x = val_x
        self.val_y = val_y
        self.val_strike = val_strike
    
    def on_epoch_end(self, epoch, logs={}):
        train_pred = self.model(train_x)
        val_pred = self.model(val_x)

        train_mse = reduce_mean(square(multiply(train_pred[:,0] - self.train_y, self.train_strike)))
        val_mse = reduce_mean(square(multiply(val_pred[:,0] - self.val_y, self.val_strike)))

        print(f' Training scaled MSE: {train_mse}, Validation scaled MSE: {val_mse}')


In [12]:
# Calculate the training and validation MSE loss on the actual option price when using price/strike as the target
def MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike):
    train_pred = model(train_x)
    val_pred = model(val_x)

    train_mse = reduce_mean(square((train_pred[:,0] - train_y)*train_strike))
    val_mse = reduce_mean(square((val_pred[:,0] - val_y)*val_strike))

    print(f' Training scaled MSE: {train_mse}, Validation scaled MSE: {val_mse}')

In [13]:
import gc
from tensorflow.keras import backend as k

class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

## Creating trainer function

In [14]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None, checkpoint_path = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # Build model and create callbacks
        model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )

        # Checkpoints
        
        # Check if the checkpoint folder exists
        if checkpoint_path and not os.path.exists(checkpoint_path):
            # Create the checkpoint folder if it does not exist
            os.makedirs(checkpoint_path)
        
        checkpoint = ModelCheckpoint(
            filepath=checkpoint_path,
            monitor='val_loss',
            mode='min',
            save_best_only=True,
            save_weights_only=True
        )

        # Adapt sequence length to config
        train_x_adjusted = [train_x[0][:, :config.LSTM_timesteps, :], train_x[1]]
        val_x_adjusted = [val_x[0][:, :config.LSTM_timesteps, :], val_x[1]]
        print(f'Train shape: {train_x_adjusted[0].shape}, {train_x_adjusted[0].shape}')
        print(f'Val shape: {val_x_adjusted[0].shape}, {val_x_adjusted[0].shape}')

        # Train model
        model.fit(
            train_x_adjusted,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x_adjusted, val_y),
            epochs = 1000,
            callbacks = [early_stopping, wandb_callback, checkpoint, ClearMemory()] if checkpoint_path else [early_stopping, wandb_callback, ClearMemory()],
        )

        if moneyness:
            MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike)

### Run full sweep

In [ ]:
#wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing - test area', count = 100)

### Single run

# Rolling window

In [15]:
def calculate_error(predictions, original):
    m = MeanSquaredError()
    m.update_state(predictions, original)
    print("MSE:", m.result().numpy())
    m = RootMeanSquaredError()
    m.update_state(predictions, original)
    print("RMSE:", m.result().numpy())

class config_object:
    def __init__(self, config):
        self.LSTM_units = config['LSTM_units']
        self.Interface_units = config['Interface_units']
        self.MLP_units = config['MLP_units']
        self.LSTM_timesteps = config['LSTM_timesteps']
        self.LSTM_layers = config['LSTM_layers']
        self.MLP_layers = config['MLP_layers']
        self.Bn_momentum = config['Bn_momentum']
        self.Lr = config['Lr']
        self.Lr_decay = config['Lr_decay']
        self.Minibatch_size = config['Minibatch_size']
        self.Min_delta = config['Min_delta']
        self.Patience = config['Patience']
        self.Num_features = config['Num_features']
        self.Architecture = config['Architecture']
        

In [18]:
num_windows = 12

# Update to v.5.0
config = {
    'LSTM_units': 4,
    'Interface_units': 32,
    'MLP_units': 600,
    'LSTM_timesteps': 10,
    'LSTM_layers': 2,
    'MLP_layers': 7,
    'Bn_momentum': 0.99,
    'Lr': 0.0010401686452862443,
    'Lr_decay': 0.800108103409341,
    'Minibatch_size': 4096,
    'Min_delta': 0.01 if moneyness else 1,
    'Patience': 20,
    'Num_features': 3 if moneyness else 4,
    'Architecture': 'LSTM-MLP v.5.0',
}

# Ask before training, so that you don't have to verify later
if google_colab == True:
  from google.colab import drive
  drive.mount('/content/drive')

df_test_combined = pd.DataFrame()

checkpoint_time = datetime.now().strftime("%m-%d_%H-%M")

for window in range(num_windows):
    if moneyness:
        train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike, = create_rw_dataset(window)
    else:
        train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset(window)

    checkpoint_path = f'./checkpoint/{checkpoint_time}/{train_start}/'

    config['Dataset'] = f'{train_start} - {val_start} - {test_start}'

    trainer(config = config, project = 'Deep learning for option pricing - rolling windows', checkpoint_path = checkpoint_path)
    co = config_object(config)
    c_model = create_model(co)
    c_model.load_weights(checkpoint_path)
    predictions = np.array(c_model(test_x))
    print(f'--- Predictions for test_start {test_start} ---')
    calculate_error(predictions, test_y)
    print('-------------------------------------------')
    df_test["Prediction"] = predictions
    df_test_combined = pd.concat([df_test_combined, df_test[["Quote_date", "Price", "Prediction"] + bs_vars]])


print(f"--- All model predictions ---")
calculate_error(df_test_combined["Prediction"], df_test_combined["Price"])
print("-------------------------------------------")

if google_colab == False:
    predictions_path = './predictions/'
    if checkpoint_path and not os.path.exists(predictions_path):
        os.makedirs(predictions_path)
    df_test_combined.to_csv(f'{predictions_path}{datetime.now().strftime("%m-%d_%H-%M")}.csv')

if google_colab == True:
    from google.colab import drive
    path = '/content/drive/My Drive/Predictions/predictions2022v.5.0.csv'
    with open(path, 'w', encoding = 'utf-8-sig') as f:
        df_test_combined.to_csv(f)

if google_colab == True:
  from google.colab import drive
  path = '/content/drive/My Drive/Predictions/predictions2022v.5.0.0.csv'
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    df_test_combined.to_csv(f)

Mounted at /content/drive


wandb: Currently logged in as: vinje (avogadro). Use `wandb login --relogin` to force relogin


Train shape: (1655728, 10, 1), (1655728, 4)
Val shape: (184525, 10, 1), (184525, 4)
Test shape: (165072, 10, 1), (165072, 4)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 10, 1)]      0           []                               
                                                                                                  
 sequential (Sequential)        (None, 32)           4832        ['input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]',                
                                                                  'input_1[0][0]']                
                                                                                              

epoch,▁
loss,▁
val_loss,▁
best_epoch,0
best_val_loss,553268.75
epoch,0
loss,463743.65625
val_loss,553268.75


...

<ipython-input-18-73059b9abbcf>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["Prediction"] = predictions


Train shape: (1758602, 10, 1), (1758602, 4)
Val shape: (144106, 10, 1), (144106, 4)
Test shape: (162934, 10, 1), (162934, 4)


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 10, 1)]      0           []                               
                                                                                                  
 sequential_1 (Sequential)      (None, 32)           4832        ['input_3[0][0]',                
                                                                  'input_3[0][0]',                
                                                                  'input_3[0][0]',                
                                                                  'input_3[0][0]',                
                                                                  'input_3[0][0]']                
                                                                                            

wandb: ERROR Control-C detected -- Run data was not synced


KeyboardInterrupt: ignored

### Load single model

In [ ]:
# Update to v.5.0
config = {
    'LSTM_units': 4,
    'Interface_units': 32,
    'MLP_units': 600,
    'LSTM_timesteps': 10,
    'LSTM_layers': 2,
    'MLP_layers': 7,
    'Bn_momentum': 0.99,
    'Lr': 0.0010401686452862443,
    'Lr_decay': 0.800108103409341,
    'Minibatch_size': 4096,
    'Min_delta': 0.01 if moneyness else 1,
    'Patience': 20,
    'Num_features': 3 if moneyness else 4,
    'Architecture': 'LSTM-MLP v.5.0',
}

window = 1
if moneyness:
        train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike, = create_rw_dataset(window)
else:
    train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset(window)

checkpoint_path = f'./checkpoint/03-20_12-35/{train_start}/'

co = config_object(config)
c_model = create_model(co)
c_model.load_weights(checkpoint_path)
predictions = np.array(c_model(test_x))
print(f'--- Predictions for {test_start} ---')
calculate_error(predictions, test_y)
print('-------------------------------------------')